In [38]:
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
import os,datetime,json
import numpy as np
#from aind_data_schema.visualizations import plot_session
import subprocess
from aind_metadata_mapper.bergamo.session import ( BergamoEtl, 
                                                  JobSettings,
                                                  RawImageInfo,
                                                 )

def extract_behavior(mouse_name, session_folder,staging_dir):
    command = ['export_behavior.bat',  mouse_name, session_folder,staging_dir]

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)

    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())

    rc = process.poll()
    return rc
mouse_ID_dict = {'BCI66': 683842,
                 'BCI67': 677017,
                 'BCI68': 685615,
                 'BCI69': 676337,
                 'BCI70': 685613,
                 'BCI71': 685611,
                 'BCI72': 688405,
                 'BCI73': 687399,
                 'BCI74': 695205,
                 'BCI75': 686681,
                 'BCI76': 698158,
                 'BCI77': 687408,
                 'BCI78': 705363,
                 'BCI79': 698175,
                 'BCI80': 705360,
                 'BCI81': 708792,
                 'BCI82': 706604,
                 'BCI83': 711604,
                 'BCI84': 715261}

# steps for synchronization
- main script only has aind libraries
- inputs are: mouse BCI name, session folder
- This script will invoke a shell script, that will
    - activate bci analysis environment and create behavior
- then generates rig and session metadata using the behavior file and tiff files
- get behavior video in a folder structure
- generates watchdog manifest file

In [ ]:
# add inputs
mouse_name = 'BCI75'
session_folder = r"F:/BCI/{mouse_name}/062824".format(mouse_name=mouse_name)

imaging_laser_wavelength = 920

platform = 'single-plane-ophys'
s3_bucket = 'aind-ophys-data'

In [35]:

labtracks_id = mouse_ID_dict[mouse_name]


if not Path(session_folder).is_dir():
    print('2p session folder not found')
    #ERROROUT
session_date = datetime.strptime(Path(session_folder).name,'%m%d%y').date()
staging_dir = r"F:/Staging/{mouse_name}_{session_date}".format(mouse_name=mouse_name,session_date = session_date)

#make folders
#Path(staging_dir).mkdir(parents=True, exist_ok=True) # this will make the staging folder
#Path(os.path.join(staging_dir,'behavior')).mkdir(parents=True, exist_ok=True) # this will make the staging folder
# ... ophys, behavior video

extract_behavior(mouse_name, session_folder,staging_dir)
behavior_fname = f"{Path(session_folder).name}-bpod_zaber.npy"

2p session folder not found


In [ ]:
# create session json from bpod dict, tiff folder and user input
bpod_dict = np.load(os.path.join(staging_dir,'behavior',behavior_fname),allow_pickle = True).tolist()
if 'scanimage_tiff_headers' not in bpod_dict.keys():
    print('no scanimage header found in behavior file - redo behavior extraction')
    #ERROROUT
    

# COPY OVER RAW BPOD FILES - TRANSCRIBE TO WINDOWS
bpod_file_names = np.unique(bpod_dict['bpod_file_names'])    
command_list = []
for f in bpod_file_names:
    copy_command = 'gsutil -m cp -r {} {} '.format('/home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/'+f[:-4],
                                             str(modality1_source)+'/')
    command_list.append(copy_command)

bash_command = r" && ".join(command_list)
#os.system(bash_command)
if not only_csv_metadata:
    for bash_command in command_list:
        subprocess.run(bash_command,shell=True)    
# COPY OVER RAW BPOD FILES - TRANSCRIBE TO WINDOWS


    
print('extracting metadata for {}'.format(behavior_dir.split('/')[-2]))
# find 

        
    
    behavior_files = os.listdir(behavior_dir)
    for behavior_file in behavior_files:
        if'bpod_zaber.npy' in behavior_file:
            behavior_data=np.load(os.path.join(behavior_dir,behavior_file),allow_pickle = True).tolist()
            break

    for r,sfn in zip(behavior_data['reward_L'],behavior_data['scanimage_file_names']):
        if type(sfn) == str:
            goodtrials.append(False)
        else:
            goodtrials.append(True)
        if len(r)==0:
            hittrials.append(False)
        else:
            hittrials.append(True)
    goodtrials = np.asarray(goodtrials)    
    hittrials = np.asarray(hittrials)    
    
    for movienames in behavior_data['behavior_movie_name_list'][goodtrials]:
        for moviename in movienames:
            if 'side' in moviename:
                is_side_camera_active = True
            if 'bottom' in moviename:
                is_bottom_camera_active = True
            if is_bottom_camera_active and is_side_camera_active:
                break
        if is_bottom_camera_active and is_side_camera_active:
                break
    cn_num = []
    for cn_now in behavior_data['scanimage_roi_outputChannelsRoiNames']:
        cn_num.append(len(cn_now))
    cn_num = int(np.mean(np.asarray(cn_num)[goodtrials]))
    if cn_num ==1:
        behavior_task_name = "single neuron BCI conditioning"
    elif cn_num ==2:
        behavior_task_name = "two neuron BCI conditioning"
    elif cn_num>2:
        behavior_task_name = "multi-neuron BCI conditioning"
    else:
        print('how many CNs??')
        asdasd
    print(behavior_task_name)
    #ry:
    imaging_laser_wavelength = 920
    for wl in imaging_wavelength_dict.keys():
        if int(session_row['subject_id']) in imaging_wavelength_dict[wl]:
            imaging_laser_wavelength = int(wl)
            print('imaging wavelength is {} nm'.format(wl))
    user_settings = JobSettings(input_source=Path(ophys_dir),
                                output_directory=Path('/'.join(behavior_dir.split('/')[:-1])),
                                experimenter_full_name=["Kayvon Daie", "Marton Rozsa"],
                                subject_id=str(int(session_row['subject_id'])),
                                imaging_laser_wavelength = imaging_laser_wavelength,
                                fov_imaging_depth= 200,
                                fov_targeted_structure= 'Primary Motor Cortex',
                                notes= 'This metadata was generated from old data post-hoc on {} - version {}'.format(datetime.now(),version),
                                session_type= "BCI",
                                iacuc_protocol=  "2109",
                                rig_id=  "442_Bergamo_2p_photostim",
                                behavior_camera_names= np.asarray(["Side Face Camera","Bottom Face Camera"])[np.asarray([is_side_camera_active,is_bottom_camera_active])].tolist(),
                                imaging_laser_name= "Chameleon Laser",
                                photostim_laser_name= "Monaco Laser",
                                photostim_laser_wavelength=  1035,
                                starting_lickport_position= [0,
                                                             -1*np.abs(np.median(behavior_data['zaber_reward_zone']-behavior_data['zaber_limit_far'])),
                                                             0],
                                behavior_task_name=  behavior_task_name,
                                hit_rate_trials_0_10= np.nanmean(hittrials[goodtrials][:10]),
                                hit_rate_trials_20_40=  np.nanmean(hittrials[goodtrials][20:40]),
                                total_hits= sum(hittrials[goodtrials]),
                                average_hit_rate=  sum(hittrials[goodtrials])/sum(goodtrials),
                                trial_num=sum(goodtrials))
    etl_job = BergamoEtl(job_settings=user_settings,)
    session_metadata = etl_job.run_job()
#     try:
#         session_metadata = etl_job.run_job()
#     except:
#         print('error during generating metadata.. skipping')
        
    # except:
    #     print('error, could not extract metadata')
    
            
    #asd



In [ ]:
# create 

    #ERROROUT
    

        
        modality0 = 'ophys'
        modality0_source = os.path.join(raw_imaging_path,mouse_id,session)
        
        
        #copy behavior stuff in a folder
        modality1 = 'trained_behavior'
        modality1_source = Path(os.path.join(CO_save_path,'{}-{}'.format(mouse_id,session),'behavior'))
        modality1_source.mkdir(parents=True, exist_ok=True)
        copy_command = 'gsutil cp {} {} '.format(behavior_fname,str(modality1_source)+'/'+f"{session}-bpod_zaber.npy")
        #reply = os.system(copy_command)
        if not only_csv_metadata:
            subprocess.run(copy_command,shell=True)
        bpod_file_names = np.unique(bpod_dict['bpod_file_names'])
        
        command_list = []
        for f in bpod_file_names:
            copy_command = 'gsutil -m cp -r {} {} '.format('/home/jupyter/bucket/Data/Behavior/raw/KayvonScope/BCI/experiments/BCI/setups/KayvonScope/sessions/'+f[:-4],
                                                     str(modality1_source)+'/')
            command_list.append(copy_command)

        bash_command = r" && ".join(command_list)
        #os.system(bash_command)
        if not only_csv_metadata:
            for bash_command in command_list:
                subprocess.run(bash_command,shell=True)

        
        #copy camera data in a single folder under side & bottom
        modality2 = 'behavior_videos'
        modality2_source = Path(os.path.join(CO_save_path,'{}-{}'.format(mouse_id,session),'behavior_videos'))
        modality2_source.mkdir(parents=True, exist_ok=True)
        modality2_source_side = Path(os.path.join(CO_save_path,'{}-{}'.format(mouse_id,session),'behavior_videos','side'))
        modality2_source_side.mkdir(parents=True, exist_ok=True)
        modality2_source_bottom = Path(os.path.join(CO_save_path,'{}-{}'.format(mouse_id,session),'behavior_videos','bottom'))
        modality2_source_bottom.mkdir(parents=True, exist_ok=True)
        
        side_folders = []
        bottom_folders = []
        
        for m in bpod_dict['behavior_movie_name_list']:
            if type(m) == np.ndarray:
                for movie_name in m:
                    if 'side' in movie_name:
                        side_folders.append(os.path.join('/home/jupyter/bucket/Data/Behavior_videos/raw/Bergamo-2P-Photostim',*movie_name.split('/')[5:-1]))
                    elif 'bottom' in movie_name:
                        bottom_folders.append(os.path.join('/home/jupyter/bucket/Data/Behavior_videos/raw/Bergamo-2P-Photostim',*movie_name.split('/')[5:-1]))
                    else:
                        wtf
        side_folders = np.unique(side_folders)
        command_list = []
        for s in side_folders:
            dest  = Path(os.path.join(modality2_source_side,s.split('/')[-1]))
            dest.mkdir(parents=True, exist_ok=True)
            copy_command = 'gsutil -m rsync {} {} '.format(s,
                                                     str(dest)+'/')
            command_list.append(copy_command)
        bottom_folders = np.unique(bottom_folders)
        for b in bottom_folders:
            dest  = Path(os.path.join(modality2_source_bottom,b.split('/')[-1]))
            dest.mkdir(parents=True, exist_ok=True)
            copy_command = 'gsutil -m rsync {} {} '.format(b,
                                                     str(dest)+'/')
            command_list.append(copy_command)

        bash_command = r" && ".join(command_list)
        #os.system(bash_command)
        if not only_csv_metadata:
            for bash_command in command_list:
                subprocess.run(bash_command,shell=True)
        #asd
        
        
        upload_dict['platform'].append(platform)
        upload_dict['acq_datetime'].append(acq_datetime)
        upload_dict['subject_id'].append(subject_id)
        upload_dict['s3_bucket'].append(s3_bucket)
        upload_dict['modality0'].append(modality0)
        upload_dict['modality0.source'].append(str(modality0_source))
        upload_dict['modality1'].append(modality1)
        upload_dict['modality1.source'].append(str(modality1_source))
        upload_dict['modality2'].append(modality2)
        upload_dict['modality2.source'].append(str(modality2_source))
        upload_dict['version'].append(version)
        
        # save this stuff
        output_df = pd.DataFrame.from_dict(upload_dict)
        for r_ in output_df.iterrows():
            pass # get the last row
        r_dict = r_[1].to_dict()
        import json
        with open(upload_json_location, 'w') as f:
            json.dump(r_dict, f)
        